# API

Raw code snippets for api part

In [68]:
import numpy as np
import uvicorn
import mlflow
import mdsist.util as util

from fastapi import FastAPI
from pydantic import BaseModel

# Load packages
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

from mdsist.dataset import MdsistDataset
from mdsist.config import DATA_DIR, PROCESSED_DATA_DIR
from mdsist.predictor import Predictor

import asyncio

import mlflow
import mlflow.pytorch

import mdsist.util as util
import json
import requests

from dotenv import load_dotenv

# Load envionment variables
load_dotenv()

True

## Implementation

In [69]:
## Import model from MLFlow, with below URI
MODEL_URI = 'runs:/10cb51b288134c48835a8c0b9fe66eca/model_20240930190709'
#'models:/CNNv1-production/'

device = util.get_available_device()

model = mlflow.pytorch.load_model(MODEL_URI, map_location=device)

pred = Predictor(model)

In [70]:
app = FastAPI()

In [71]:
class PredictionRequest(BaseModel):
    image: list # The input data should be an array list of 256 pixel


@app.get("/test")
async def test():
    return "It works"

@app.post("/mnist-model-prediction/")
async def predict(data: PredictionRequest):
    # pass the image as byte, then from buffer.
    
    image_array = np.frombuffer(dtype= np.uint8) # this method interprets a buffer as a 1D array. 

    # reshape uint8 to below structure
    # shape (N, 1, H, W) # N = degree of freedom, -1.  H = 28, W = 28

    images = image_array.reshape((-1, 1, 28, 28))
    
    prediction = pred.predict(images)

    print(prediction)
    #return{"prediction": int(prediction[0])}


## API test

In [72]:

with open(DATA_DIR /'images_jpg/test/0/13.jpg', 'rb') as p:
    image = p.read()

In [73]:
test_dataset = MdsistDataset(PROCESSED_DATA_DIR / 'test.parquet')
image_dict = test_dataset.data.loc[0, "image"]
image_array = test_dataset.decode_png_image(image_dict)
image = image_dict.get('bytes')

In [74]:
response = requests.get("http://localhost:8000/test")
print(json.loads(response.text))

It works


for some reason, image shape is not coincident

In [75]:
# Set the headers to indicate the content type as 'application/octet-stream'
headers = {'Content-Type': 'application/octet-stream'}
resp = requests.post("http://localhost:8000/mnist-model-prediction", headers=headers,data=image)
print(resp.text)

Internal Server Error
